In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
df_x=  pd.read_csv('C:/Users/bhara/Downloads/x_train.csv')
df_y =  pd.read_csv('C:/Users/bhara/Downloads/y_train.csv')
df_x_test =  pd.read_csv('C:/Users/bhara/Downloads/x_test.csv')
x_input_text = df_x['text']
XTest = df_x_test['text']
Y = df_y['is_positive_sentiment'].values

Y = Y.reshape((Y.shape[0],1))
print(x_input_text)
print(XTest)
print(Y)
Y.shape

0       Oh and I forgot to also mention the weird colo...
1                            THAT one didn't work either.
2                                      Waste of 13 bucks.
3       Product is useless, since it does not have eno...
4       None of the three sizes they sent with the hea...
                              ...                        
2395    The sweet potato fries were very good and seas...
2396    I could eat their bruschetta all day it is dev...
2397                                 Ambience is perfect.
2398    We ordered the duck rare and it was pink and t...
2399         Service was good and the company was better!
Name: text, Length: 2400, dtype: object
0      It only recognizes the Phone as its storage de...
1      Disappointing accessory from a good manufacturer.
2      The one big drawback of the MP3 player is that...
3      This particular model would not work with my M...
4      If the two were seperated by a mere 5+ ft I st...
                             ...     

(2400, 1)

In [99]:
from collections import OrderedDict


word_embeddings = pd.read_csv('C:/Users/bhara/Downloads/glove.6B.50d.txt.zip',
                               header=None, sep=' ', index_col=0,
                               nrows=100000, compression='zip', encoding='utf-8', quoting=3)
word_list = word_embeddings.index.values.tolist()
glove = OrderedDict(zip(word_list, word_embeddings.values))

In [100]:
def get_sentence_embedding(sentence):
    words = sentence.split()
    embeddings = []
    for word in words:
        if word in glove:
            embeddings.append(glove[word])
    if len(embeddings) == 0:
        return np.zeros(50)
    else:
        return np.mean(embeddings, axis=0)

In [101]:
from torch.autograd import Variable
X = np.array([get_sentence_embedding(sentence) for sentence in x_input_text])
X_full = Variable(torch.from_numpy(X)).float()
Y_full = Variable(torch.from_numpy(Y)).float()
print(X_full.shape)
print(Y_full.shape)


torch.Size([2400, 50])
torch.Size([2400, 1])


In [102]:
import torch.nn.functional as F
from sklearn.model_selection import KFold

class LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.linear = nn.Linear(input_size, 1)
    
    def forward(self, x):
        return F.sigmoid(self.linear(x))
    
loss_func=[]
accuracy_list = []
folds = 10
learning_rate = 0.2
num_epochs = 20000

model = LogisticRegression(X_full.shape[1])
criterion = torch.nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters())    
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

kf = KFold(n_splits=folds, shuffle=True)


In [103]:
for fold, (train_index_val, test_index_val) in enumerate(kf.split(X_full)):
    X_train, X_test = X_full[train_index_val], X_full[test_index_val]
    y_train, y_test = Y_full[train_index_val], Y_full[test_index_val]
    
    for iteration in range(num_epochs):
        output = model(X_train)
        label = y_train.view(-1, 1) 

        loss = criterion(output, label)
        loss_func.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        output = model(X_test)
        ypred = (output >= 0.5).float()
        accuracy = (ypred == y_test.view(-1, 1)).float().mean().item()
        accuracy_list.append(accuracy)
    print(sum(loss_func)/len(loss_func))
print(accuracy_list)

accuracy_avg = sum(accuracy_list) / folds

print(f"Average Accuracy = ", accuracy_avg)


tensor(0.5776, grad_fn=<DivBackward0>)
tensor(0.5795, grad_fn=<DivBackward0>)
tensor(0.5800, grad_fn=<DivBackward0>)
tensor(0.5793, grad_fn=<DivBackward0>)
tensor(0.5778, grad_fn=<DivBackward0>)
tensor(0.5779, grad_fn=<DivBackward0>)
tensor(0.5779, grad_fn=<DivBackward0>)
tensor(0.5780, grad_fn=<DivBackward0>)
tensor(0.5780, grad_fn=<DivBackward0>)
tensor(0.5780, grad_fn=<DivBackward0>)
[0.6583333611488342, 0.7083333134651184, 0.699999988079071, 0.7083333134651184, 0.6541666388511658, 0.6958333253860474, 0.6625000238418579, 0.7291666865348816, 0.6708333492279053, 0.6333333253860474]
Average Accuracy =  0.6820833325386048
